In [1]:
# Change the CWD to be the root so that the giles_conf.json file is found in the correct location
import os
os.chdir("../../../")

In [2]:
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format

In [3]:
import pandas as pd
import emission.core.get_database as edb
import bson.objectid as boi
import numpy as np
import scipy.interpolate as spi
import arrow

In [4]:
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.storage.timeseries.abstract_timeseries as esta
import emission.core.wrapper.entry as ecwe
import emission.core.wrapper.location as ecwl
import emission.analysis.intake.cleaning.location_smoothing as eaicl

In [5]:
import emission.analysis.plotting.geojson.geojson_feature_converter as gfc
import emission.analysis.plotting.leaflet_osm.ipython_helper as ipy
import emission.analysis.plotting.leaflet_osm.our_plotter as lo

In [6]:
edb.get_timeseries_db().distinct('user_id')

[UUID('c528bcd2-a88b-3e82-be62-ef4f2396967a')]

In [7]:
curr_id = edb.get_timeseries_db().distinct('user_id')[0]
trips_df = esda.get_data_df(esda.RAW_TRIP_KEY, curr_id, time_query=None)

In [8]:
trips_df[["_id", "start_fmt_time", "start_ts", "end_fmt_time", "end_ts", "start_place"]]

,_id,start_fmt_time,start_ts,end_fmt_time,end_ts,start_place
0,57bdd2aff6858f83e9b6c4a6,2016-02-22T08:58:38.727512-08:00,1456160318.727512,2016-02-22T09:02:37.006508-08:00,1456160557.006508,57bdd2aff6858f83e9b6c4a5
1,57bdd2aff6858f83e9b6c4a8,2016-02-22T10:06:20.206589-08:00,1456164380.206589,2016-02-22T10:07:00.000028-08:00,1456164420.000028,57bdd2aff6858f83e9b6c4a7
2,57bdd2aff6858f83e9b6c4aa,2016-02-22T10:27:11.000019-08:00,1456165631.000019,2016-02-22T10:30:15.000020-08:00,1456165815.000020,57bdd2aff6858f83e9b6c4a9
3,57bdd2aff6858f83e9b6c4ac,2016-02-22T13:07:22.999994-08:00,1456175242.999994,2016-02-22T13:13:16.999949-08:00,1456175596.999949,57bdd2aff6858f83e9b6c4ab
4,57bdd2aff6858f83e9b6c4ae,2016-02-22T15:35:58.097793-08:00,1456184158.097793,2016-02-22T15:37:35.000060-08:00,1456184255.000060,57bdd2aff6858f83e9b6c4ad
5,57bdd2aff6858f83e9b6c4b0,2016-02-22T15:49:46.999964-08:00,1456184986.999964,2016-02-22T16:04:02.000039-08:00,1456185842.000039,57bdd2aff6858f83e9b6c4af
6,57bdd2aff6858f83e9b6c4b2,2016-02-22T16:49:34.000114-08:00,1456188574.000114,2016-02-22T16:57:04.000037-08:00,1456189024.000037,57bdd2aff6858f83e9b6c4b1
7,57bdd2aff6858f83e9b6c4b4,2016-02-22T19:40:59.136611-08:00,1456198859.136611,2016-02-22T19:48:14.999756-08:00,1456199294.999756,57bdd2aff6858f83e9b6c4b3
8,57bdd2aff6858f83e9b6c4b6,2016-02-22T19:58:56.999930-08:00,1456199936.999930,2016-02-22T20:11:47.000001-08:00,1456200707.000001,57bdd2aff6858f83e9b6c4b5


In [9]:
import emission.storage.decorations.trip_queries as esdt

We are going to focus on the trips highlighted in the bug https://github.com/e-mission/e-mission-server/issues/288

## Short trips highlighted in the bug

### Trip from Kaiser to Cognition (\_id = 57bb7d6af6858f2284bb4682)

In [10]:
kctrip = trips_df.iloc[1] 

In [11]:
kcid = kctrip._id

In [12]:
section_entry = esdt.get_raw_sections_for_trip(curr_id, kcid)[0]

In [13]:
section_entry.data.start_loc, section_entry.data.start_fmt_time

({"coordinates": [-122.0835895956217, 37.38701632412202], "type": "Point"},
 u'2016-02-22T10:06:20.206589-08:00')

In [14]:
ts = esta.TimeSeries.get_time_series(curr_id)

In [15]:
# loc_entry_df.append(loc_entry_df).reset_index()[["fmt_time", "ts"]]

In [16]:
loc_entry_it = ts.find_entries(["background/filtered_location"],
                                   esda.get_time_query_for_trip_like(
                                       esda.RAW_SECTION_KEY, section_entry.get_id()))
loc_entry_list = [ecwe.Entry(e) for e in loc_entry_it]
[(e.data.ts, e.data.fmt_time, e.data.loc.coordinates) for e in loc_entry_list]

[(1456164380.206589,
  u'2016-02-22T10:06:20.206589-08:00',
  [-122.0835895956217, 37.38701632412202]),
 (1456164381.940137,
  u'2016-02-22T10:06:21.940137-08:00',
  [-122.0832249430214, 37.38680795769895]),
 (1456164420.000028,
  u'2016-02-22T10:07:00.000028-08:00',
  [-122.0834267792498, 37.38637934908027])]

#### 2-D interpolation

In [17]:
if len(loc_entry_list) < 4:
    loc_entry_list.extend(loc_entry_list)

In [18]:
len(loc_entry_list)

6

In [19]:
loc_entry_df = pd.DataFrame([ts._to_df_entry(e) for e in loc_entry_list])

In [20]:
time_fn = spi.interp2d(np.array(loc_entry_df.latitude), np.array(loc_entry_df.longitude), np.array(loc_entry_df.ts))

In [21]:
start_place = ts.get_entry_from_id(esda.RAW_PLACE_KEY, trips_df.iloc[1].start_place)

In [22]:
start_place

Entry({u'data': {u'enter_fmt_time': u'2016-02-22T09:02:37.006508-08:00', u'exit_local_dt': {u'hour': 10, u'month': 2, u'second': 20, u'weekday': 0, u'year': 2016, u'timezone': u'America/Los_Angeles', u'day': 22, u'minute': 6}, u'exit_fmt_time': u'2016-02-22T10:06:20.206589-08:00', u'enter_local_dt': {u'hour': 9, u'month': 2, u'second': 37, u'weekday': 0, u'year': 2016, u'timezone': u'America/Los_Angeles', u'day': 22, u'minute': 2}, u'ending_trip': ObjectId('57bdd2aff6858f83e9b6c4a6'), u'starting_trip': ObjectId('57bdd2aff6858f83e9b6c4a8'), u'source': u'DwellSegmentationDistFilter', u'location': {u'type': u'Point', u'coordinates': [-122.0812194754829, 37.38939195184179]}, u'enter_ts': 1456160557.006508, u'duration': 3823.200080871582, u'exit_ts': 1456164380.206589}, u'_id': ObjectId('57bdd2aff6858f83e9b6c4a7'), u'user_id': UUID('c528bcd2-a88b-3e82-be62-ef4f2396967a'), u'metadata': {u'write_fmt_time': u'2016-08-24T10:00:31.303532-07:00', u'write_ts': 1472058031.303532, u'time_zone': u'Am

In [23]:
arrow.get(time_fn(start_place.data.location.coordinates[1], start_place.data.location.coordinates[0])).to("America/Los_Angeles")

<Arrow [2012-08-30T01:38:01.451709-07:00]>

In [24]:
arrow.get(time_fn(start_place.data.location.coordinates[0], start_place.data.location.coordinates[1])).to("America/Los_Angeles")

<Arrow [2023-04-21T17:09:20.374315-07:00]>

#### Doesn't work!!

#### Use average time and distance to start to estimate travel time

In [25]:
import emission.analysis.intake.cleaning.clean_and_resample as eaicc
import emission.analysis.point_features as pf
import emission.core.common as ecc

In [26]:
with_speeds_df = eaicl.add_dist_heading_speed(loc_entry_df)

In [27]:
with_speeds_df_nona = with_speeds_df.dropna()

In [28]:
with_speeds_df.speed.median()

1.336997664083294

In [29]:
start_place = ts.get_entry_from_id(esda.RAW_PLACE_KEY, kctrip.start_place)

In [30]:
[loc_entry_df.iloc[0].longitude, loc_entry_df.iloc[0].latitude], start_place.data.location.coordinates

([-122.0835895956217, 37.387016324122023],
 [-122.0812194754829, 37.38939195184179])

In [31]:
add_dist = ecc.calDistance(loc_entry_list[0].data.loc.coordinates, start_place.data.location.coordinates)
print add_dist

337.085318098


In [32]:
add_time = add_dist / with_speeds_df.speed.median()
add_time

252.12109725653383

In [33]:
(40 + add_time) / 60

4.868684954275564

In [34]:
75 + add_dist

412.0853180981027

### Close enough to the real values (418m/3 mins)

### Trip from home to the bookstore (\_id = 57bb7d67f6858f2284bb465d)

In [35]:
hbtrip = trips_df.iloc[-2]

In [36]:
hbid = hbtrip._id

In [37]:
len(esdt.get_raw_sections_for_trip(curr_id, hbid))

3

In [38]:
for s in esdt.get_raw_sections_for_trip(curr_id, hbid):
    print s.data.start_fmt_time, s.data.end_fmt_time, s.data.sensed_mode, s.data.start_stop

2016-02-22T19:40:59.136611-08:00 2016-02-22T19:41:02.000037-08:00 MotionTypes.BICYCLING None
2016-02-22T19:41:41.000092-08:00 2016-02-22T19:41:41.000092-08:00 MotionTypes.RUNNING 57bdd2b0f6858f83e9b6c4c6
2016-02-22T19:42:16.000092-08:00 2016-02-22T19:48:14.999756-08:00 MotionTypes.WALKING 57bdd2b0f6858f83e9b6c4c8


In [39]:
section_entry = esdt.get_raw_sections_for_trip(curr_id, hbid)[0]

In [40]:
section_entry.data.start_loc, section_entry.data.start_fmt_time

({"coordinates": [-122.083723212413, 37.39098770265726], "type": "Point"},
 u'2016-02-22T19:40:59.136611-08:00')

In [41]:
ts = esta.TimeSeries.get_time_series(curr_id)

In [42]:
# loc_entry_df.append(loc_entry_df).reset_index()[["fmt_time", "ts"]]

In [43]:
loc_entry_it = ts.find_entries(["background/filtered_location"],
                                   esda.get_time_query_for_trip_like(
                                       esda.RAW_SECTION_KEY, section_entry.get_id()))
loc_entry_list = [ecwe.Entry(e) for e in loc_entry_it]
[(e.data.ts, e.data.fmt_time, e.data.loc.coordinates) for e in loc_entry_list]

[(1456198859.136611,
  u'2016-02-22T19:40:59.136611-08:00',
  [-122.083723212413, 37.39098770265726]),
 (1456198862.000037,
  u'2016-02-22T19:41:02.000037-08:00',
  [-122.0837376640384, 37.39093872721995])]

#### 2-D interpolation

In [44]:
if len(loc_entry_list) < 4:
    loc_entry_list.extend(loc_entry_list)

In [45]:
loc_entry_df = pd.DataFrame([ts._to_df_entry(e) for e in loc_entry_list])

In [46]:
np.array(loc_entry_df.latitude), np.array(loc_entry_df.longitude), np.array(loc_entry_df.ts)

(array([ 37.3909877 ,  37.39093873,  37.3909877 ,  37.39093873]),
 array([-122.08372321, -122.08373766, -122.08372321, -122.08373766]),
 array([  1.45619886e+09,   1.45619886e+09,   1.45619886e+09,
          1.45619886e+09]))

In [47]:
len(np.array(loc_entry_df.latitude)), len(np.array(loc_entry_df.longitude)), len(np.array(loc_entry_df.ts))

(4, 4, 4)

In [48]:
time_fn = spi.interp2d(np.array(loc_entry_df.latitude), np.array(loc_entry_df.longitude), np.array(loc_entry_df.ts))

In [49]:
start_place = ts.get_entry_from_id(esda.RAW_PLACE_KEY, hbtrip.start_place)

In [50]:
start_place

Entry({u'data': {u'enter_fmt_time': u'2016-02-22T16:57:04.000037-08:00', u'exit_local_dt': {u'hour': 19, u'month': 2, u'second': 59, u'weekday': 0, u'year': 2016, u'timezone': u'America/Los_Angeles', u'day': 22, u'minute': 40}, u'exit_fmt_time': u'2016-02-22T19:40:59.136611-08:00', u'enter_local_dt': {u'hour': 16, u'month': 2, u'second': 4, u'weekday': 0, u'year': 2016, u'timezone': u'America/Los_Angeles', u'day': 22, u'minute': 57}, u'ending_trip': ObjectId('57bdd2aff6858f83e9b6c4b2'), u'starting_trip': ObjectId('57bdd2aff6858f83e9b6c4b4'), u'source': u'DwellSegmentationDistFilter', u'location': {u'type': u'Point', u'coordinates': [-122.0864089802965, 37.39077901331944]}, u'enter_ts': 1456189024.000037, u'duration': 9835.136574029922, u'exit_ts': 1456198859.136611}, u'_id': ObjectId('57bdd2aff6858f83e9b6c4b3'), u'user_id': UUID('c528bcd2-a88b-3e82-be62-ef4f2396967a'), u'metadata': {u'write_fmt_time': u'2016-08-24T10:00:31.465567-07:00', u'write_ts': 1472058031.465567, u'time_zone': u'

In [51]:
start_place.data.location

{"coordinates": [-122.0864089802965, 37.39077901331944], "type": "Point"}

In [52]:
arrow.get(time_fn(start_place.data.location.coordinates[1], start_place.data.location.coordinates[0])).to("America/Los_Angeles")

<Arrow [2016-02-22T19:41:02.000037-08:00]>

In [53]:
arrow.get(time_fn(start_place.data.location.coordinates[0], start_place.data.location.coordinates[1])).to("America/Los_Angeles")

<Arrow [1969-12-31T16:00:00-08:00]>

#### Doesn't work!! (19:41:02) is after the current trip start

#### Use average time and distance to start to estimate travel time

In [54]:
import emission.analysis.intake.cleaning.clean_and_resample as eaicc
import emission.analysis.point_features as pf
import emission.core.common as ecc

In [55]:
loc_entry_it = ts.find_entries(["background/filtered_location"],
                                   esda.get_time_query_for_trip_like(
                                       esda.RAW_SECTION_KEY, section_entry.get_id()))
loc_entry_list = [ecwe.Entry(e) for e in loc_entry_it]
loc_entry_df = pd.DataFrame([ts._to_df_entry(e) for e in loc_entry_list])
loc_entry_df[["ts", "fmt_time", "latitude", "longitude"]]

,ts,fmt_time,latitude,longitude
0,1456198859.136611,2016-02-22T19:40:59.136611-08:00,37.390988,-122.083723
1,1456198862.000037,2016-02-22T19:41:02.000037-08:00,37.390939,-122.083738


In [56]:
with_speeds_df = eaicl.add_dist_heading_speed(loc_entry_df)

In [57]:
with_speeds_df.speed, with_speeds_df.sensed_speed

(0   0.000000
 1   1.953422
 Name: speed, dtype: float64, 0   -1.000000
 1    0.830000
 Name: sensed_speed, dtype: float64)

In [58]:
with_speeds_df_nona = with_speeds_df.dropna()

In [59]:
with_speeds_df.speed

0   0.000000
1   1.953422
Name: speed, dtype: float64

In [60]:
with_speeds_df.speed.median()

0.9767110416278179

In [61]:
start_place = ts.get_entry_from_id(esda.RAW_PLACE_KEY, hbtrip.start_place)

In [62]:
[loc_entry_list[0].data.loc.coordinates], start_place.data.location.coordinates

([[-122.083723212413, 37.39098770265726]],
 [-122.0864089802965, 37.39077901331944])

In [63]:
add_dist = ecc.calDistance(loc_entry_list[0].data.loc.coordinates, start_place.data.location.coordinates)
print add_dist

238.407845367


speed ($s$) is in m/s. dist ($d$) is in meters.
so what we really need is

| Distance (m) | Time (s) | 
| --- | ---- |
| $s$ m | 1 sec |
| $d$ m | ? secs = $\frac{d}{s}$ |

In [64]:
add_time = add_dist / with_speeds_df.speed.median()
add_time

244.09250556802144

In [65]:
(7 * 60  + add_time) / 60

11.06820842613369

In [66]:
356 + add_dist

594.4078453668861

### Closer to the real values (737m/13 mins), unsure if close enough

## Hooking up the results to the standard data structure

In [67]:
ts_delta = add_dist * with_speeds_df.speed.median()

In [68]:
loc_entry_it = ts.find_entries(["background/filtered_location"],
                                   esda.get_time_query_for_trip_like(
                                       esda.RAW_SECTION_KEY, section_entry.get_id()))
loc_entry_list = [ecwe.Entry(e) for e in loc_entry_it]
loc_entry_df = pd.DataFrame([ts._to_df_entry(e) for e in loc_entry_list])
loc_entry_df[["ts", "fmt_time", "latitude", "longitude"]]

,ts,fmt_time,latitude,longitude
0,1456198859.136611,2016-02-22T19:40:59.136611-08:00,37.390988,-122.083723
1,1456198862.000037,2016-02-22T19:41:02.000037-08:00,37.390939,-122.083738


In [69]:
loc_entry_df.columns

Index([              u'_id',          u'accuracy',          u'altitude',
                  u'filter',             u'floor',          u'fmt_time',
                 u'heading',          u'latitude',               u'loc',
            u'local_dt_day',     u'local_dt_hour',   u'local_dt_minute',
          u'local_dt_month',   u'local_dt_second', u'local_dt_timezone',
        u'local_dt_weekday',     u'local_dt_year',         u'longitude',
       u'metadata_write_ts',      u'sensed_speed',                u'ts',
                 u'user_id',         u'vaccuracy'],
      dtype='object')

In [70]:
start_place_enter_loc_entry = ecwe.Entry(ts.get_entry_at_ts("background/filtered_location", "data.ts", start_place.data.enter_ts))

In [71]:
start_place_enter_loc_entry

Entry({u'user_id': UUID('c528bcd2-a88b-3e82-be62-ef4f2396967a'), u'_id': ObjectId('56cbcb48eaedff78c76364a1'), u'data': {u'loc': {u'type': u'Point', u'coordinates': [-122.0864089802965, 37.39077901331944]}, u'fmt_time': u'2016-02-22T16:57:04.000037-08:00', u'altitude': 23.24042189585946, u'floor': 0, u'vaccuracy': 3, u'ts': 1456189024.000037, u'longitude': -122.0864089802965, u'filter': u'distance', u'local_dt': {u'hour': 16, u'month': 2, u'second': 4, u'weekday': 0, u'year': 2016, u'timezone': u'America/Los_Angeles', u'day': 22, u'minute': 57}, u'latitude': 37.39077901331944, u'heading': 211.2995758056641, u'sensed_speed': 2.700000047683716, u'accuracy': 5}, u'metadata': {u'write_fmt_time': u'2016-02-22T16:57:04.281355-08:00', u'plugin': u'none', u'write_ts': 1456189024.281355, u'time_zone': u'America/Los_Angeles', u'platform': u'ios', u'write_local_dt': {u'hour': 16, u'month': 2, u'second': 4, u'weekday': 0, u'year': 2016, u'timezone': u'America/Los_Angeles', u'day': 22, u'minute': 5

In [72]:
loc_entry_list.insert(0, start_place_enter_loc_entry)

In [73]:
loc_entry_df = pd.DataFrame([ts._to_df_entry(e) for e in loc_entry_list])
loc_entry_df[["ts", "fmt_time", "latitude", "longitude"]]

,ts,fmt_time,latitude,longitude
0,1456189024.000037,2016-02-22T16:57:04.000037-08:00,37.390779,-122.086409
1,1456198859.136611,2016-02-22T19:40:59.136611-08:00,37.390988,-122.083723
2,1456198862.000037,2016-02-22T19:41:02.000037-08:00,37.390939,-122.083738


In [74]:
new_start_ts = loc_entry_df.loc[1].ts - ts_delta
new_start_ts, arrow.get(new_start_ts).to("America/Los_Angeles")

(1456198626.2810359, <Arrow [2016-02-22T19:37:06.281036-08:00]>)

In [75]:
loc_entry_df.ts.iloc[0] = new_start_ts
loc_entry_df.fmt_time.iloc[0] = arrow.get(new_start_ts).to(start_place_enter_loc_entry.data.local_dt.timezone)

/Users/shankari/OSS/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [76]:
loc_entry_df

,_id,accuracy,altitude,filter,floor,fmt_time,heading,latitude,loc,local_dt_day,...,local_dt_second,local_dt_timezone,local_dt_weekday,local_dt_year,longitude,metadata_write_ts,sensed_speed,ts,user_id,vaccuracy
0,56cbcb48eaedff78c76364a1,5,23.240422,distance,0,2016-02-22T19:37:06.281036-08:00,211.299576,37.390779,"{u'type': u'Point', u'coordinates': [-122.0864...",22,...,4,America/Los_Angeles,0,2016,-122.086409,1456189024.281355,2.700000,1456198626.281036,c528bcd2-a88b-3e82-be62-ef4f2396967a,3.000000
1,56cbf56beaedff78c76369ec,65,33.000000,distance,0,2016-02-22T19:40:59.136611-08:00,-1.000000,37.390988,"{u'type': u'Point', u'coordinates': [-122.0837...",22,...,59,America/Los_Angeles,0,2016,-122.083723,1456198860.176294,-1.000000,1456198859.136611,c528bcd2-a88b-3e82-be62-ef4f2396967a,62.250000
2,56cbf56beaedff78c76369ee,10,33.781250,distance,0,2016-02-22T19:41:02.000037-08:00,158.554688,37.390939,"{u'type': u'Point', u'coordinates': [-122.0837...",22,...,2,America/Los_Angeles,0,2016,-122.083738,1456198862.140979,0.830000,1456198862.000037,c528bcd2-a88b-3e82-be62-ef4f2396967a,12.000000


## Sanity testing some longer trips (to and from Stevenson in the afternoon)

### To Stevenson (around 1pm)

In [77]:
trip = trips_df.iloc[3] 

In [78]:
trip_id = trip._id

In [79]:
len(esdt.get_raw_sections_for_trip(curr_id, trip_id))

1

In [80]:
section_entry = esdt.get_raw_sections_for_trip(curr_id, trip_id)[0]

In [81]:
section_entry.data.start_loc, section_entry.data.start_fmt_time

({"coordinates": [-122.0814648271744, 37.39806556612011], "type": "Point"},
 u'2016-02-22T13:07:22.999994-08:00')

In [82]:
ts = esta.TimeSeries.get_time_series(curr_id)

In [83]:
loc_entry_it = ts.find_entries(["background/filtered_location"],
                                   esda.get_time_query_for_trip_like(
                                       esda.RAW_SECTION_KEY, section_entry.get_id()))
loc_entry_list = [ecwe.Entry(e) for e in loc_entry_it]
loc_entry_df = pd.DataFrame([ts._to_df_entry(e) for e in loc_entry_list])
loc_entry_df[["ts", "fmt_time", "latitude", "longitude"]]

,ts,fmt_time,latitude,longitude
0,1456175242.999994,2016-02-22T13:07:22.999994-08:00,37.398066,-122.081465
1,1456175245.999949,2016-02-22T13:07:25.999949-08:00,37.398509,-122.081235
2,1456175248.999949,2016-02-22T13:07:28.999949-08:00,37.398987,-122.081067
3,1456175252.999949,2016-02-22T13:07:32.999949-08:00,37.399448,-122.080887
4,1456175262.999950,2016-02-22T13:07:42.999950-08:00,37.399812,-122.081338
5,1456175269.999950,2016-02-22T13:07:49.999950-08:00,37.399919,-122.081979
6,1456175275.999949,2016-02-22T13:07:55.999949-08:00,37.400036,-122.082636
7,1456175279.999950,2016-02-22T13:07:59.999950-08:00,37.400158,-122.083314
8,1456175283.999949,2016-02-22T13:08:03.999949-08:00,37.400282,-122.083989
9,1456175287.999949,2016-02-22T13:08:07.999949-08:00,37.400391,-122.084554


In [84]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(loc_entry_df)])])

Found 18 features from 18 points


In [85]:
with_speeds_df = eaicl.add_dist_heading_speed(loc_entry_df)

In [86]:
with_speeds_df.speed, with_speeds_df.sensed_speed

(0     0.000000
 1    17.776499
 2    18.393565
 3    13.410949
 4     5.674975
 5     8.267077
 6     9.921079
 7    15.335081
 8    15.317327
 9    12.824680
 10    4.314984
 11    8.048916
 12   10.494730
 13   11.080352
 14   11.046274
 15    4.251954
 16    0.235353
 17    1.068487
 Name: speed, dtype: float64, 0    15.010000
 1    16.480000
 2    16.139999
 3    14.350000
 4     6.510000
 5     9.130000
 6    10.440000
 7    13.940000
 8    14.660000
 9    13.220000
 10    3.570000
 11    8.230000
 12   10.840000
 13   10.780000
 14    9.910000
 15    2.410000
 16    1.050000
 17    1.280000
 Name: sensed_speed, dtype: float64)

In [87]:
with_speeds_df_nona = with_speeds_df.dropna()

In [88]:
with_speeds_df.speed

0     0.000000
1    17.776499
2    18.393565
3    13.410949
4     5.674975
5     8.267077
6     9.921079
7    15.335081
8    15.317327
9    12.824680
10    4.314984
11    8.048916
12   10.494730
13   11.080352
14   11.046274
15    4.251954
16    0.235353
17    1.068487
Name: speed, dtype: float64

In [89]:
with_speeds_df.speed.median()

10.207904896126383

In [90]:
start_place = ts.get_entry_from_id(esda.RAW_PLACE_KEY, trip.start_place)

In [91]:
[loc_entry_list[0].data.loc.coordinates], start_place.data.location.coordinates

([[-122.0814648271744, 37.39806556612011]],
 [-122.0865191986059, 37.39084250297154])

In [92]:
add_dist = ecc.calDistance(loc_entry_list[0].data.loc.coordinates, start_place.data.location.coordinates)
print add_dist

918.939745549


speed ($s$) is in m/s. dist ($d$) is in meters.
so what we really need is

| Distance (m) | Time (s) | 
| --- | ---- |
| $s$ m | 1 sec |
| $d$ m | ? secs = $\frac{d}{s}$ |

In [93]:
add_time = add_dist / with_speeds_df.speed.median()
add_time

90.02236550007575

In [94]:
section_entry.data.duration/60, (section_entry.data.duration  + add_time) / 60

(5.899999248981476, 7.400372007316071)

In [95]:
with_speeds_df_nona.distance.sum(), with_speeds_df_nona.distance.sum() + add_dist

(944.35994144333768, 1863.2996869924398)

In [96]:
arrow.get(start_place.data.exit_ts - add_time).to("America/Los_Angeles")

<Arrow [2016-02-22T13:05:52.977628-08:00]>

### From Stevenson (around 3:30pm)

In [97]:
trip = trips_df.iloc[4] 

In [98]:
trip_id = trip._id

In [99]:
len(esdt.get_raw_sections_for_trip(curr_id, trip_id))

1

In [100]:
section_entry = esdt.get_raw_sections_for_trip(curr_id, trip_id)[0]

In [101]:
section_entry.data.start_loc, section_entry.data.start_fmt_time

({"coordinates": [-122.0835446168422, 37.39569463873612], "type": "Point"},
 u'2016-02-22T15:35:58.097793-08:00')

In [102]:
ts = esta.TimeSeries.get_time_series(curr_id)

In [103]:
loc_entry_it = ts.find_entries(["background/filtered_location"],
                                   esda.get_time_query_for_trip_like(
                                       esda.RAW_SECTION_KEY, section_entry.get_id()))
loc_entry_list = [ecwe.Entry(e) for e in loc_entry_it]
loc_entry_df = pd.DataFrame([ts._to_df_entry(e) for e in loc_entry_list])
loc_entry_df[["ts", "fmt_time", "latitude", "longitude"]]

,ts,fmt_time,latitude,longitude
0,1456184158.097793,2016-02-22T15:35:58.097793-08:00,37.395695,-122.083545
1,1456184158.441907,2016-02-22T15:35:58.441907-08:00,37.395695,-122.083545
2,1456184160.401226,2016-02-22T15:36:00.401226-08:00,37.395923,-122.083234
3,1456184183.000059,2016-02-22T15:36:23.000059-08:00,37.395530,-122.083529
4,1456184189.000060,2016-02-22T15:36:29.000060-08:00,37.395106,-122.083726
5,1456184194.000060,2016-02-22T15:36:34.000060-08:00,37.394660,-122.083997
6,1456184199.000059,2016-02-22T15:36:39.000059-08:00,37.394221,-122.084270
7,1456184203.000060,2016-02-22T15:36:43.000060-08:00,37.393812,-122.084526
8,1456184207.000059,2016-02-22T15:36:47.000059-08:00,37.393357,-122.084809
9,1456184211.000059,2016-02-22T15:36:51.000059-08:00,37.392901,-122.085094


In [104]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(loc_entry_df)])])

Found 16 features from 16 points


In [105]:
with_speeds_df = eaicl.add_dist_heading_speed(loc_entry_df)

In [106]:
with_speeds_df.speed, with_speeds_df.sensed_speed

(0     0.000000
 1     0.000000
 2    19.080527
 3     2.253770
 4     8.376666
 5    11.015684
 6    10.877709
 7    12.701925
 8    14.100501
 9    14.157825
 10   14.501768
 11   15.375246
 12   14.146435
 13   11.074563
 14   11.416149
 15    2.390169
 Name: speed, dtype: float64, 0    -1.000000
 1    -1.000000
 2     0.520000
 3     9.420000
 4     8.620000
 5    10.600000
 6    11.150000
 7    12.980000
 8    13.400000
 9    14.280000
 10   14.810000
 11   14.580000
 12   14.430000
 13    9.870000
 14    2.420000
 15    1.230000
 Name: sensed_speed, dtype: float64)

In [107]:
with_speeds_df_nona = with_speeds_df.dropna()

In [108]:
with_speeds_df.speed

0     0.000000
1     0.000000
2    19.080527
3     2.253770
4     8.376666
5    11.015684
6    10.877709
7    12.701925
8    14.100501
9    14.157825
10   14.501768
11   15.375246
12   14.146435
13   11.074563
14   11.416149
15    2.390169
Name: speed, dtype: float64

In [109]:
with_speeds_df.speed.median()

11.245356183166976

In [110]:
start_place = ts.get_entry_from_id(esda.RAW_PLACE_KEY, trip.start_place)

In [111]:
[loc_entry_list[0].data.loc.coordinates], start_place.data.location.coordinates

([[-122.0835446168422, 37.39569463873612]],
 [-122.0841398277526, 37.40355206656959])

In [112]:
add_dist = ecc.calDistance(loc_entry_list[0].data.loc.coordinates, start_place.data.location.coordinates)
print add_dist

875.286713476


speed ($s$) is in m/s. dist ($d$) is in meters.
so what we really need is

| Distance (m) | Time (s) | 
| --- | ---- |
| $d$ = 11.24 m | 1 sec |
| $d$ = 875.28 m | ? secs = $\frac{d}{s}$ $\approx$ 77|

In [113]:
add_time = add_dist / with_speeds_df.speed.median()
add_time

77.83539260285401

In [114]:
section_entry.data.duration/60, (section_entry.data.duration  + add_time) / 60

(1.6150377829869589, 2.912294326367859)

In [115]:
with_speeds_df_nona.distance.sum(), with_speeds_df_nona.distance.sum() + add_dist

(762.04245176805932, 1637.3291652437929)

In [116]:
arrow.get(start_place.data.exit_ts - add_time).to("America/Los_Angeles")

<Arrow [2016-02-22T15:34:40.262400-08:00]>

### Hm, the new duration after everything is just 2.9 $\approx$ 3 mins! Seems way too short compared to the trip to Stevenson

It is because in the trip to Stevenson, although we arrive at the campus by 13:08, we continue to get points until 13:13 as we walk to the classroom. But here, points end at 15:37 and then there's a large gap until we leave for karate at 15:49-ish. If we had got a couple of more points, we would be up there on this too. The bigger issue is that we don't have points for half the trip on each side, and while going to school, the missed points are mostly on a straight line down Shoreline while coming back from school, the missed points are on the Wright curve.

In [125]:
loc_entry_df = ts.get_data_df("background/filtered_location", time_query=None)
loc_entry_df[(loc_entry_df.local_dt_hour==15) & (loc_entry_df.local_dt_minute < 55)][["ts", "fmt_time", "latitude", "longitude"]]

,ts,fmt_time,latitude,longitude
43,1456184158.097793,2016-02-22T15:35:58.097793-08:00,37.395695,-122.083545
44,1456184158.441907,2016-02-22T15:35:58.441907-08:00,37.395695,-122.083545
45,1456184160.401226,2016-02-22T15:36:00.401226-08:00,37.395923,-122.083234
46,1456184183.000059,2016-02-22T15:36:23.000059-08:00,37.395530,-122.083529
47,1456184189.000060,2016-02-22T15:36:29.000060-08:00,37.395106,-122.083726
48,1456184194.000060,2016-02-22T15:36:34.000060-08:00,37.394660,-122.083997
49,1456184199.000059,2016-02-22T15:36:39.000059-08:00,37.394221,-122.084270
50,1456184203.000060,2016-02-22T15:36:43.000060-08:00,37.393812,-122.084526
51,1456184207.000059,2016-02-22T15:36:47.000059-08:00,37.393357,-122.084809
52,1456184211.000059,2016-02-22T15:36:51.000059-08:00,37.392901,-122.085094


## Dataframe testing

In [117]:
test_df = pd.DataFrame({"A": [1,2,3], "B": [11,12,13]})
test_df

,A,B
0,1,11
1,2,12
2,3,13


In [118]:
test_df.loc[-1] = [0, 10]
test_df

,A,B
0,1,11
1,2,12
2,3,13
-1,0,10


In [119]:
test_df.sort_index().reset_index(drop=True)

,A,B
0,0,10
1,1,11
2,2,12
3,3,13


In [120]:
x = [1,2,3]

In [121]:
x.pop(0)
x

[2, 3]